In [21]:
%gui qt5
from py4j.java_gateway import JavaGateway
from napari import ViewerApp
import time, os
import numpy as np
from PyQt5.QtCore import QThreadPool, QRunnable
from pycudadecon import decon, make_otf
import tifffile

import warnings
warnings.filterwarnings('ignore')

In [9]:
gate = JavaGateway()
viewer = ViewerApp()


In [10]:
layer1 = viewer.add_image(np.memmap('Q:\Snap-Live-Stream.dat',
                                    dtype=np.uint16, offset=0,
                                    mode='r',
                                    shape=(2048, 2048)))


In [28]:
file = 'Q:\Snap-Live-Stream.dat'
psf_path = "C:\\Data\\Bryant\\PSFs\\488_64x64_2D.tif"

In [12]:
class ProcessRunnable(QRunnable):
    def __init__(self, target, args):
        QRunnable.__init__(self)
        self.t = target
        self.args = args

    def run(self):
        self.t(*self.args)

    def start(self):
        QThreadPool.globalInstance().start(self)



In [16]:
def decon_gpu(image, psf_path):
    return decon(image, psf_path)

def stream_decon():
    count = 0
    while True:
        time.sleep(0.001)
        layer1.image = decon_gpu(
            np.memmap('Q:\Snap-Live-Stream.dat',
                      dtype=np.uint16, offset=0,
                      shape=(2048, 2048)),
            psf_path)
        if count >= 10000:
            break
        count += 1


In [7]:
r = ProcessRunnable(target=stream_decon, args=())
r.start()

In [35]:
result = decon(np.memmap('Q:\Snap-Live-Stream.dat',
                      dtype=np.uint16, offset=0,
                      shape=(2048, 2048)), psf_path)

ValueError: not enough values to unpack (expected 3, got 2)

In [36]:
result = np.asarray(np.memmap('Q:\Snap-Live-Stream.dat',
                      dtype=np.uint16, offset=0,
                      shape=(2048, 2048)))

In [44]:
psf = tifffile.imread(psf_path)


In [45]:
out = decon( result, psf_path)

ValueError: not enough values to unpack (expected 3, got 2)